In [2]:
# Importing related module
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
from chromedriver_py import binary_path
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
options = webdriver.ChromeOptions()
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from pandas.core.frame import DataFrame
from bs4 import BeautifulSoup as Soup
browser = webdriver.Chrome(service=Service(executable_path=binary_path), options=options)
browser.get("https://www.instagram.com/")

# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_elements(By.NAME,'username')[0]
password_input = browser.find_elements(By.NAME,'password')[0]
print("inputing username and password...")

# ------ 輸入帳號密碼 ------
username_input.send_keys("") #輸入帳號
password_input.send_keys("") #輸入密碼
time.sleep(3)
# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="loginForm"]/div/div[3]/button/div')))
# ------ 網頁元素定位 ------
login_click = browser.find_elements(By.XPATH,'//*[@id="loginForm"]/div/div[3]/button/div')[0]
# ------ 點擊登入鍵 ------
login_click.click()
time.sleep(5)

# ------ 不儲存登入資料 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div/div/div/div')))

# ------網頁元素定位 ------
store_click = WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "x1i10hfl")))

print("not storing finding...")
# ------ 點擊不儲存鍵 ------
store_click.click()
print("not storing successfully!")

# ------ 不開啟通知 ------ #xpath可能不同帳號不同
notification_click = WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "_a9--")))
print("not notification finding...")

# ------ 點擊不開啟通知 ------
notification_click.click()
print("not notification successfully!")

time.sleep(3)
# 爬取短連結
url = 'https://www.instagram.com/explore/locations/602043149991850/'#我要爬的網址
browser.get(url)

n_scroll = 15
post_url = []

for i in range(n_scroll):
    scroll = 'window.scrollTo(0, document.body.scrollHeight);'
    browser.execute_script(scroll)
    markup = browser.page_source
    soup = Soup(markup, "html.parser")

    for elem in soup.select('article div div div div a'):
        if elem['href'] not in post_url:
            post_url.append(elem['href'])

    time.sleep(10)

print("總共取得 " + str(len(post_url)) + " 篇貼文連結")

# 創建兩個列表來存儲文章內容和讚數
post_contents = []
likes_count = []

# 遍歷每個貼文的短網址
for short_url in post_url:
    full_url = f'https://www.instagram.com{short_url}'
    browser.get(full_url)
    #爬取貼文
    try:
        post_element = browser.find_element(By.CSS_SELECTOR, "span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp.xo1l8bm.x5n08af.x10wh9bi.x1wdrske.x8viiok.x18hxmgj")
        post_content = post_element.text
    except:
        post_content = "can't found"
    #爬取讚數
    try:
        subs_element = browser.find_element(By.CSS_SELECTOR, "span.html-span.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1hl2dhg.x16tdsg8.x1vvkbs")
        subs_num = subs_element.text
    except:
        subs_num = "can't found"
    
    # 將文章內容和讚數加入各自的列表
    post_contents.append(post_content)
    likes_count.append(subs_num)
    time.sleep(10)
# 結果
for i in range(len(post_contents)):
    print(f"文章內容: {post_contents[i]}, 讚數: {likes_count[i]}")

browser.quit()

from pandas.core.frame import DataFrame
import pandas as pd
# 創建 DataFrame
data = {'文章內容': post_contents, '讚數': likes_count}
df = pd.DataFrame(data)

# 保存為 CSV 文件
df.to_csv('instagram_data_tea.csv', index=False)

# 打印 DataFrame
print(df)